### GARCH(1,1) Volatility forecast

In [77]:
import sys
!{sys.executable} -m pip install numpy pandas numpy openpyxl scipy arch | grep -v 'already satisfied'

import pandas as pd
import numpy as np
import openpyxl
from arch import arch_model
from arch.__future__ import reindexing

from scipy.stats import t

In [78]:
df = pd.read_excel("Data/Data.xlsx", sheet_name="df_sp500")

df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.columns = ["Date", "Open", "High", "Low","Close", "Volume"]
df["Returns"]=df['Close'].pct_change()*100

df.dropna(inplace=True)

df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

df["vol"] = df["Returns"].abs()

cols=["Date", "Returns", "vol"]
Data = df[cols]

Data.head()

Data

,Date,Returns,vol
0,2018-06-12,0.174339,0.174339
1,2018-06-13,-0.402613,0.402613
2,2018-06-14,0.247155,0.247155
3,2018-06-15,-0.101710,0.101710
4,2018-06-18,-0.212613,0.212613
...,...,...,...
1253,2023-06-05,-0.200358,0.200358
1254,2023-06-06,0.235390,0.235390
1255,2023-06-07,-0.381201,0.381201
1256,2023-06-08,0.618864,0.618864


In [79]:
#Define rolling window size
window_size = 252

#DataFrame to save volatility estimates
results = pd.DataFrame(index=Data.index)

#GARCH(1,1) rolling 252, and save in data
for start in range(0, len(Data) - window_size):
    end = start + window_size
    # Select the window of returns
    window_returns = Data['Returns'].iloc[start:end]
    # Define the GARCH(1,1) model
    model = arch_model(window_returns, vol='Garch', p=1, q=1)
    # Fit the GARCH(1,1) model
    model_fit = model.fit(disp='off')
    # Predict the next volatility
    pred = model_fit.forecast(start=0)
    # Store the volatility estimate
    results.loc[df.index[end], 'GARCH_vol'] = np.sqrt(pred.variance.values[-1, :])
    # Store the GARCH(1,1) parameters
    results.loc[df.index[end], 'omega'] = model_fit.params['omega']
    results.loc[df.index[end], 'alpha'] = model_fit.params['alpha[1]']
    results.loc[df.index[end], 'beta'] = model_fit.params['beta[1]']

In [80]:
results["gamma"]=1-results["alpha"]-results["beta"]
results["sqrt(VL)"]=np.sqrt(results["omega"]/(1-results["alpha"]-results["beta"]))

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [81]:
results

,GARCH_vol,omega,alpha,beta,gamma,sqrt(VL)
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1253,0.882759,1.740989e-08,0.000000e+00,0.994656,0.005344,0.001805
1254,0.877562,1.724018e-08,0.000000e+00,0.994632,0.005368,0.001792
1255,0.865425,1.713892e-08,3.384429e-12,0.994503,0.005497,0.001766
1256,0.842665,1.713632e-08,0.000000e+00,0.994057,0.005943,0.001698


In [82]:
Final_Data = pd.merge(Data, results, left_index=True, right_index=True, how='inner')

In [83]:
Final_Data.tail()

,Date,Returns,vol,GARCH_vol,omega,alpha,beta,gamma,sqrt(VL)
1253,2023-06-05,-0.200358,0.200358,0.882759,1.740989e-08,0.000000e+00,0.994656,0.005344,0.001805
1254,2023-06-06,0.235390,0.235390,0.877562,1.724018e-08,0.000000e+00,0.994632,0.005368,0.001792
1255,2023-06-07,-0.381201,0.381201,0.865425,1.713892e-08,3.384429e-12,0.994503,0.005497,0.001766
1256,2023-06-08,0.618864,0.618864,0.842665,1.713632e-08,0.000000e+00,0.994057,0.005943,0.001698
1257,2023-06-09,0.114806,0.114806,0.825677,1.711610e-08,0.000000e+00,0.993707,0.006293,0.001649
